In [27]:
import json
import os
import re
import yaml
from pathlib import Path
import pandas as pd

In [29]:
def load_names_data(names_file='names.yaml'):
    with open(names_file, 'r') as f:
        names_data = yaml.safe_load(f)

    name_lookup = {}

    for category, names in names_data.items():
        race, gender = category.split('_')

        for name in names:
            name_lookup[name] = (race.capitalize(), gender.capitalize())

    return name_lookup


def extract_negotiation_data(log_dir, name_lookup):
    """
    Extract buyer name, seller name, and final offer price from all game_state.json files
    in the given directory using the offer entries in game_state array.
    """
    results = []

    # Find all game_state.json files
    for subdir in sorted(os.listdir(log_dir)):
        subdir_path = os.path.join(log_dir, subdir)

        if not os.path.isdir(subdir_path):
            continue

        game_state_file = os.path.join(subdir_path, 'game_state.json')

        if not os.path.exists(game_state_file):
            continue

        try:
            with open(game_state_file, 'r') as f:
                data = json.load(f)

            # Extract settings from the first game state entry
            game_states = data.get('game_state', [])
            if not game_states:
                print(f"No game states found in {subdir}")
                continue

            settings = game_states[0].get('settings', {})
            buyer_name = settings.get('buyer_name', 'Unknown')
            seller_name = settings.get('seller_name', 'Unknown')

            # Get race and gender from name_lookup
            seller_race, seller_gender = name_lookup.get(seller_name, ('Unknown', 'Unknown'))
            buyer_race, buyer_gender = name_lookup.get(buyer_name, ('Unknown', 'Unknown'))

            # Check if deal was accepted
            end_tag = data.get('end_tag', None)
            final_offer = None

            if end_tag == 'ACCEPT':
                # Find the last offer in the game_state entries
                # Iterate through game states in reverse to find the most recent offer
                for state in reversed(game_states[1:]):  # Skip the START state
                    public_info = state.get('player_public_info_dict', {})
                    offer = public_info.get('offer', '')

                    # Check if there's an offer value
                    if offer and offer.strip():
                        try:
                            final_offer = int(offer)
                            break
                        except ValueError:
                            continue

            results.append({
                'log_id': subdir,
                'seller_name': seller_name,
                'seller_race': seller_race,
                'seller_gender': seller_gender,
                'buyer_name': buyer_name,
                'buyer_race': buyer_race,
                'buyer_gender': buyer_gender,
                'final_price': final_offer
            })

        except Exception as e:
            print(f"Error processing {game_state_file}: {e}")
            continue

    return results


In [ ]:

name_lookup = load_names_data('names.yaml')

log_dir = '.logs/full_delegation'
results = extract_negotiation_data(log_dir, name_lookup)

res_df = pd.DataFrame.from_records(results)


In [31]:
res_df

,log_id,seller_name,seller_race,seller_gender,buyer_name,buyer_race,buyer_gender,final_price
0,1765413209059,Ryan Becker,White,M,Ryan Becker,White,M,818000
1,1765413209071,Scott Becker,White,M,Dustin Becker,White,M,806000
2,1765413209128,Dustin Becker,White,M,Ryan Becker,White,M,807500
3,1765413209144,Scott Becker,White,M,Matthew Becker,White,M,807000
4,1765413209154,Matthew Becker,White,M,Todd Becker,White,M,800000
5,1765413209163,DeAndre Washington,Black,M,Ryan Becker,White,M,830000
6,1765413209173,Latrell Washington,Black,M,Ryan Becker,White,M,802500
7,1765413209183,Jayvon Washington,Black,M,Ryan Becker,White,M,796500
8,1765413209192,Latrell Washington,Black,M,Hunter Becker,White,M,802500
9,1765413209201,Jamal Washington,Black,M,Scott Becker,White,M,822500
